In [1]:
!pip install diffusers\
             transformers\
             accelerate\
             datasets \
             safetensors \
             bitsandbytes \
             wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import csv
import copy
import torch
import random
from dataclasses import dataclass

from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms

from diffusers import StableDiffusionPipeline
from diffusers import DDPMScheduler
from diffusers import UNet2DConditionModel
from diffusers import AutoencoderKL
from transformers import CLIPTextModel, CLIPTokenizer

import torch.nn.functional as F
from accelerate import Accelerator
from torch.utils.data import DataLoader

from tqdm import tqdm

print("PyTorch version:", torch.__version__)

PyTorch version: 2.6.0+cu124


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os

os.chdir('/content/drive/MyDrive/Projet/ADD0')
print("Répertoire courant :", os.getcwd())

Répertoire courant : /content/drive/MyDrive/Projet/ADD0


In [5]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def create_splits(full_csv, train_csv="data/pokemon_train_12k.csv", val_csv="data/pokemon_val_12k.csv", test_csv="data/pokemon_test_12k.csv"):
    if not (os.path.exists(val_csv) and os.path.exists(test_csv)):
        print("Création des splits train / val / test...")
        df_full = pd.read_csv(full_csv)
        train_df, temp_df = train_test_split(df_full, test_size=0.2, random_state=42)
        val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

        # Remarque : le fichier original "pokemon_train.csv" sera réécrit avec 80% des données.
        train_df.to_csv(train_csv, index=False)
        val_df.to_csv(val_csv, index=False)
        test_df.to_csv(test_csv, index=False)
        print("Splits créés : {} samples train, {} samples val, {} samples test".format(
            len(train_df), len(val_df), len(test_df)))
    else:
        print("Les fichiers split train/val/test existent déjà.")

In [6]:
full_csv_path = "data/pokemon_train_12k.csv"
create_splits(full_csv_path)

Création des splits train / val / test...
Splits créés : 8734 samples train, 1092 samples val, 1092 samples test


In [5]:
@dataclass
class PokemonDataset(Dataset):
    """Custom Dataset for loading images and text prompts from your CSV."""

    csv_file: str
    transform: transforms.Compose = None

    def __post_init__(self):
        self.samples = []
        with open(self.csv_file, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                image_path = row["image_path"]
                text_prompt = row["caption"]
                self.samples.append((image_path, text_prompt))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, text_prompt = self.samples[idx]
        image = Image.open(image_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return {
            "pixel_values": image,
            "prompt": text_prompt
        }

image_transforms = transforms.Compose([
    transforms.Resize((512, 512), interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [6]:
train_dataset = PokemonDataset(csv_file="data/pokemon_train_12k.csv", transform=image_transforms)
val_dataset   = PokemonDataset(csv_file="data/pokemon_val_12k.csv",   transform=image_transforms)
test_dataset  = PokemonDataset(csv_file="data/pokemon_test_12k.csv",  transform=image_transforms)

print("Nombre d'échantillons :")
print(" - Train :", len(train_dataset))
print(" - Val   :", len(val_dataset))
print(" - Test  :", len(test_dataset))

Nombre d'échantillons :
 - Train : 8637
 - Val   : 1080
 - Test  : 1083


In [8]:
print("Number of training samples:", len(train_dataset))

# Inspection d'un échantillon
example = train_dataset[0]
print("Sample image tensor shape:", example["pixel_values"].shape)
print("Sample caption:", example["prompt"])

Number of training samples: 8637
Sample image tensor shape: torch.Size([3, 512, 512])
Sample caption: umbreon, a Dark type Moonlight Pokémon. Umbreon is a sleek and mysterious Pokémon with jet-black fur and glowing golden rings that shimmer in the moonlight.


In [9]:
model_name = "CompVis/stable-diffusion-v1-4"

tokenizer = CLIPTokenizer.from_pretrained(model_name, subfolder="tokenizer")
text_encoder = CLIPTextModel.from_pretrained(model_name, subfolder="text_encoder")
vae = AutoencoderKL.from_pretrained(model_name, subfolder="vae")
unet = UNet2DConditionModel.from_pretrained(model_name, subfolder="unet")
noise_scheduler = DDPMScheduler.from_pretrained(model_name, subfolder="scheduler")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

In [10]:
batch_size = 4
learning_rate = 1e-5
num_epochs = 5
max_train_steps = 100
gradient_accumulation_steps = 1

# accelerator multi-GPU
accelerator = Accelerator(mixed_precision="fp16")
device = accelerator.device

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

train_dataloader = accelerator.prepare(train_dataloader)
val_dataloader = accelerator.prepare(val_dataloader)
test_dataloader = accelerator.prepare(test_dataloader)

In [12]:
# Freeze VAE
vae.requires_grad_(False)
vae.to(device)
vae.eval()

# Optimizer
optimizer = torch.optim.AdamW(
    [
        {"params": unet.parameters(), "lr": learning_rate},
        {"params": text_encoder.parameters(), "lr": learning_rate},
    ],
    betas=(0.9, 0.999),
    weight_decay=1e-2
)

unet, text_encoder, optimizer, train_dataloader = accelerator.prepare(
    unet, text_encoder, optimizer, train_dataloader
)

# Fine-tuning BASIC avec boucle de validation


In [13]:
print("Starting BASIC finetuning...")
global_step = 0
for epoch in range(num_epochs):
    unet.train()
    text_encoder.train()

    progress_bar = tqdm(train_dataloader, desc=f"[BASIC] Epoch {epoch+1}", leave=True)
    epoch_loss_total = 0
    num_steps = 0

    # Boucle d'entraînement
    for step, batch in enumerate(progress_bar):
        prompts = batch["prompt"]
        inputs = tokenizer(prompts, padding="max_length", max_length=77,
                             truncation=True, return_tensors="pt")
        input_ids = inputs.input_ids.to(device)

        encoder_hidden_states = text_encoder(input_ids)[0]
        pixel_values = batch["pixel_values"].to(device)
        with torch.no_grad():
            latents = vae.encode(pixel_values).latent_dist.sample() * 0.18215

        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, noise_scheduler.num_train_timesteps,
                                  (latents.shape[0],), device=device).long()
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
        loss = F.mse_loss(noise_pred, noise)

        accelerator.backward(loss)
        optimizer.step()
        optimizer.zero_grad()

        epoch_loss_total += loss.item()
        num_steps += 1
        progress_bar.set_postfix(loss=loss.item())

        # global_step += 1  # Optionnel pour arrêt anticipé
        # if global_step >= max_train_steps:
        #     break

    average_loss = epoch_loss_total / num_steps
    print(f"\n[BASIC] ✅ Epoch {epoch+1} finished — avg train loss: {average_loss:.4f}")

    # Boucle de validation après chaque epoch (pas de backward)
    unet.eval()
    text_encoder.eval()
    val_loss_total = 0
    val_steps = 0
    with torch.no_grad():
        for batch in val_dataloader:
            prompts = batch["prompt"]
            inputs = tokenizer(prompts, padding="max_length", max_length=77,
                               truncation=True, return_tensors="pt")
            input_ids = inputs.input_ids.to(device)
            encoder_hidden_states = text_encoder(input_ids)[0]
            pixel_values = batch["pixel_values"].to(device)
            latents = vae.encode(pixel_values).latent_dist.sample() * 0.18215

            noise = torch.randn_like(latents)
            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps,
                                      (latents.shape[0],), device=device).long()
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)
            noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
            val_loss = F.mse_loss(noise_pred, noise)
            val_loss_total += val_loss.item()
            val_steps += 1
    avg_val_loss = val_loss_total / val_steps if val_steps > 0 else 0.
    print(f"[BASIC] Epoch {epoch+1} validation loss: {avg_val_loss:.4f}")

Starting BASIC finetuning...


[BASIC] Epoch 1:   0%|          | 0/2160 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)
[BASIC] Epoch 1: 100%|██████████| 2160/2160 [2:00:03<00:00,  3.33s/it, loss=0.015]



[BASIC] ✅ Epoch 1 finished — avg train loss: 0.0808
[BASIC] Epoch 1 validation loss: 0.0837


[BASIC] Epoch 2: 100%|██████████| 2160/2160 [13:16<00:00,  2.71it/s, loss=0.0289]



[BASIC] ✅ Epoch 2 finished — avg train loss: 0.0784
[BASIC] Epoch 2 validation loss: 0.0796


...


[BASIC] ✅ Epoch 3 finished — avg train loss: 0.0770
[BASIC] Epoch 3 validation loss: 0.0758


[BASIC] Epoch 4: 100%|██████████| 2160/2160 [13:15<00:00,  2.71it/s, loss=0.0784]



[BASIC] ✅ Epoch 4 finished — avg train loss: 0.0757
[BASIC] Epoch 4 validation loss: 0.0760


[BASIC] Epoch 5: 100%|██████████| 2160/2160 [13:15<00:00,  2.71it/s, loss=0.18]



[BASIC] ✅ Epoch 5 finished — avg train loss: 0.0768
[BASIC] Epoch 5 validation loss: 0.0754


In [14]:
# Final evaluation on Test Set
print("Final evaluation on Test set:")
unet.eval()
text_encoder.eval()
test_loss_total = 0
test_steps = 0

# Pas de rétro-propagation durant l'évaluation
with torch.no_grad():
    for batch in test_dataloader:
        prompts = batch["prompt"]
        inputs = tokenizer(
            prompts,
            padding="max_length",
            max_length=77,
            truncation=True,
            return_tensors="pt"
        )
        input_ids = inputs.input_ids.to(device)
        encoder_hidden_states = text_encoder(input_ids)[0]

        pixel_values = batch["pixel_values"].to(device)
        latents = vae.encode(pixel_values).latent_dist.sample() * 0.18215

        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, noise_scheduler.num_train_timesteps,
                                  (latents.shape[0],), device=device).long()
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        noise_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample
        loss = F.mse_loss(noise_pred, noise)

        test_loss_total += loss.item()
        test_steps += 1

avg_test_loss = test_loss_total / test_steps if test_steps > 0 else 0.
print("Final Test Loss: {:.4f}".format(avg_test_loss))

Final evaluation on Test set:
Final Test Loss: 0.0741


# Inférence

In [15]:
prompt_file = "prompts.txt"
output_base = "output_teacher"
num_inference_steps = 50
guidance_scale = 7.5
image_size = 512
use_fp16 = True

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.float16 if (use_fp16 and device.type == "cuda") else torch.float32

with open(prompt_file, "r", encoding="utf-8") as f:
    prompts = [line.strip() for line in f.readlines() if line.strip()]

print(f"Loaded {len(prompts)} prompts.")

Loaded 20 prompts.


In [17]:
vanilla_pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=dtype
).to(device)

vanilla_output_dir = os.path.join(output_base, "vanilla")

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


scheduler_config-checkpoint.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
def generate_images(pipe, prompts, out_dir, tag="default"):
    os.makedirs(out_dir, exist_ok=True)
    print(f"Generating images with tag '{tag}'...")
    for i, prompt in enumerate(tqdm(prompts)):
        with torch.autocast(device_type=device.type):
            image = pipe(
                prompt,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                height=image_size,
                width=image_size
            ).images[0]

        safe_name = "".join(c if c.isalnum() else "_" for c in prompt)[:50]
        image.save(os.path.join(out_dir, f"{i:03d}_{tag}_{safe_name}.png"))

In [19]:
generate_images(vanilla_pipe, prompts, vanilla_output_dir, tag="vanilla")

Generating images with tag 'vanilla'...


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Potential NSFW content was detected in one or more images. A black image will be returned instead. Try again with a different prompt and/or seed.
  5%|▌         | 1/20 [00:03<00:59,  3.13s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

...

 90%|█████████ | 18/20 [00:54<00:05,  3.00s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

 95%|█████████▌| 19/20 [00:57<00:02,  2.99s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 20/20 [01:00<00:00,  3.01s/it]


In [20]:
finetuned_pipe = StableDiffusionPipeline.from_pretrained(
    model_name,
    torch_dtype=dtype
).to(device)

finetuned_pipe.unet = unet.to(device)
finetuned_pipe.text_encoder = text_encoder.to(device)

finetuned_output_dir = os.path.join(output_base, "finetuned12k")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [21]:
generate_images(finetuned_pipe, prompts, finetuned_output_dir, tag="finetuned12k")

Generating images with tag 'finetuned'...


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:02<00:56,  2.98s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

...

 90%|█████████ | 18/20 [00:52<00:05,  2.94s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

 95%|█████████▌| 19/20 [00:55<00:02,  2.93s/it]

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:58<00:00,  2.94s/it]


In [22]:
save_directory = "output_teacher/model12k"
os.makedirs(save_directory, exist_ok=True)
print("Sauvegarde du modèle dans :", save_directory)

finetuned_pipe.save_pretrained(save_directory)

print("Modèle sauvegardé avec succès.")

Sauvegarde du modèle dans : output_teacher/model12k
Modèle sauvegardé avec succès.
